### Basic TF Model & Processing Structure
- Config class with parameters
- Model class with methods for inference, train, evaluation
    - using lazy_property decorator
    
#### Goal: RNN for learning a simple real-valued function

#### Open questions
- How do Flags fit into the picture?
- and main.app()?

In [3]:
from __future__ import absolute_import, division, print_function
import os, time
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tnrange, tqdm_notebook

HOME = os.environ['HOME']
os.chdir(HOME+"/ninja/mxn/src/MxnVentures")
from mxn_ventures import utils

%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Data Generation

In [4]:

def rnn_data(data, time_steps, labels=False):
    """
    creates new data frame based on previous observation
      * example:
        l = [1, 2, 3, 4, 5]
        time_steps = 2
        -> labels == False [[1, 2], [2, 3], [3, 4]]
        -> labels == True [2, 3, 4, 5]
    """
    rnn_df = []
    for i in range(len(data) - time_steps):
        if labels:
            try:
                rnn_df.append(data.iloc[i + time_steps].as_matrix())
            except AttributeError:
                rnn_df.append(data.iloc[i + time_steps])
        else:
            data_ = data.iloc[i: i + time_steps].as_matrix()
            rnn_df.append(data_ if len(data_.shape) > 1 else [[i] for i in data_])
    return np.array(rnn_df)


def split_data(data, val_size=0.1, test_size=0.1):
    """
    splits data to training, validation and testing parts
    """
    ntest = int(round(len(data) * (1 - test_size)))
    nval = int(round(len(data.iloc[:ntest]) * (1 - val_size)))

    df_train, df_val, df_test = data.iloc[:nval], data.iloc[nval:ntest], data.iloc[ntest:]

    return df_train, df_val, df_test


def prepare_data(data, time_steps, labels=False, val_size=0.1, test_size=0.1):
    """
    Given the number of `time_steps` and some data,
    prepares training, validation and test data for an lstm cell.
    """
    df_train, df_val, df_test = split_data(data, val_size, test_size)
    return (rnn_data(df_train, time_steps, labels=labels),
            rnn_data(df_val, time_steps, labels=labels),
            rnn_data(df_test, time_steps, labels=labels))

def generate_data(fct, x, time_steps, seperate=False):
    """generates data with based on a function fct"""
    data = fct(x)
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)
    train_x, val_x, test_x = prepare_data(data['a'] if seperate else data, time_steps)
    train_y, val_y, test_y = prepare_data(data['b'] if seperate else data, time_steps, labels=True)
    return dict(train=train_x, val=val_x, test=test_x), dict(train=train_y, val=val_y, test=test_y)

In [8]:
class OrigConfig(object):
    """Original config."""
    layers = [100]      # n_layers = 1, hidden_size = 100
    n_epochs = 10       # max_max_epoch = 13
    batch_size = 50
    # probability of dropout of hidden units
    dropout_p_hidden = 0.5 # 0.4
    learning_rate = 0.01   # 0.05  # lr
    # momentum: if 0, Nesterov momentum is applied during training
    momentum = 0.0
    # learning rate adaptation strategy
    adapt = 'adagrad'
    # decay parameter for RMSProp; has no effect in other modes
    decay = 0.9
    # clip gradients above this value; 0 means no clippling
    grad_cap = 0
    # initialization width: either stdev, or min/max of the init interval;
    # 0 means adaptive normalization (sigma depends on weigth matrix size)
    sigma = 0
    # either using normal or uniform distr. for weight initialization
    init_as_normal = False
    # loss function (top1, bpr, cross-entropy=CE)¯˘
    loss = 'top1'
    # activation function for hidden cells (tanh, relu)
    hidden_act = 'tanh'
    # final layer activation function 
    # (CE: softmax; top1/bpr: tanh /relu/linear)
    final_act = None
    # either to randomize sessions order in each epoch
    train_random_order = False
    # lambda coefficient of the L2 regularization
    lmbd = 0.0
    # RNN: either to reset hidden state to zero after a session finished
    reset_after_session = True    
    # data column names (in header)
    session_key = 'SessionId'
    item_key = 'ItemId'
    time_key = 'Time'
    
    num_steps = 20 # from tf.rnn - what is it exactly?
    cell_type = 'gru' # gru, lstm
    forget_bias = 0.0
    
    log_dir = HOME + "/data/tf_tut/logs/s1"
    time_steps = 3
    rnn_layers = [{'num_units': 5}]
    dense_layers = None
    training_steps = 10000
    print_interval = 1000 # training_steps / 10
    batch_size = 100
    dtype = tf.float32
    
    def __init__(sefl):
        pass
    def info(self):
        utils.print_parameters(self)

class SmallConfig(OrigConfig):
    """Small config."""
    layers = [5]
    n_epochs = 2

In [ ]:
class Model1(object):
    # based on Tut2 PTBModel
    def __init__(self, config):
        self.batch_size = config.batch_size
        lstm_layers = []
        for layer in config.rnn_layers:
            num_neurons = layer['num_units']
            cell = tf.nn.rnn_cell.BasicLSTMCell(num_neurons, state_is_tuple=True) 
            if 'keep_prob' in layer:
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, layer['keep_prob'])
            lstm_layers.append(layer)
        lstm_net = tf.nn.rnn_cell.MultiRNNCell(lstm_layers, state_is_tuple=True)

        self.data = tf.placeholder(config.dtype, [self.batch_size, config.time_steps])
        self.target = tf.placeholder(config.dtype, [self.batch_size, config.time_steps])

        output, state = tf.nn.dynamic_rnn(lstm_net, self.data, dtype=config.dtype)
        
        prediction, loss = tf.contrib.learn.models.linear_regression(output, self.target)
        train_op = tf.contrib.layers.optimize_loss(loss, tf.contrib.layers.optimize_loss(
            loss, tf.contrib.framework.get_global_step(), optimizer=optimizer, learning_rate=learning_rate))

        
        self._input_data = tf.placeholder(tf.int32, [self.batch_size, self.num_steps])
        self._targets = tf.placeholder(tf.int32, [self.batch_size, self.num_steps])
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0, state_is_tuple=True)



In [ ]:
class Model:
    def __init__(self, config, data, target):
        self.config = config
        self.data = data
        self.target = target
        self.prediction
        self.optimize
        self.error
    @utils.lazy_property
    def prediction(self):
        data_size = int(self.data.get_shape()[1])
        target_size = int(self.target.get_shape()[1])
        weight = tf.Variable(tf.truncated_normal([data_size, target_size]))
        bias = tf.Variable(tf.constant(0.1, shape=[target_size]))
        incoming = tf.matmul(self.data, weight) + bias
        return tf.nn.softmax(incoming)
    @utils.lazy_property
    def optimize(self):
        cross_entropy = -tf.reduce_sum(self.target, tf.log(self.prediction))
        optimizer = tf.train.RMSPropOptimizer(0.03)
        return optimizer.minimize(cross_entropy)
    @utils.lazy_property
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))
